In [9]:
pip install numpy pandas scikit-learn matplotlib seaborn tensorflow keras Pillow

     ---------------------------------------- 0.0/46.1 kB ? eta -:--:--
     ---------------------------------------- 46.1/46.1 kB ? eta 0:00:00
   ---------------------------------------- 0.0/377.1 MB ? eta -:--:--
   ---------------------------------------- 1.0/377.1 MB 20.9 MB/s eta 0:00:18
   ---------------------------------------- 3.3/377.1 MB 35.8 MB/s eta 0:00:11
    --------------------------------------- 4.8/377.1 MB 34.5 MB/s eta 0:00:11
    --------------------------------------- 7.5/377.1 MB 39.8 MB/s eta 0:00:10
    --------------------------------------- 9.4/377.1 MB 40.2 MB/s eta 0:00:10
   - -------------------------------------- 11.4/377.1 MB 43.5 MB/s eta 0:00:09
   - -------------------------------------- 13.3/377.1 MB 43.5 MB/s eta 0:00:09
   - -------------------------------------- 15.3/377.1 MB 46.7 MB/s eta 0:00:08
   - -------------------------------------- 17.4/377.1 MB 40.9 MB/s eta 0:00:09
   -- ------------------------------------- 19.3/377.1 MB 40.9 MB/s e

Code heavily referenced from ChatGPT, GitHub, GeekforGeeks, and StackOverflow.

Part A - Support Vector Machine

1. Train a support vector machine using the images from fer2013.csv. Use the Training set for training, and the PrivateTest test set for testing. Report precision, recall, accuracy, F1 score, and create a confusion matrix on the test set, showing the confusions between emotion labels.

In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix

# https://pandas.pydata.org/
# https://numpy.org/
# https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html
# https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html
# https://matplotlib.org/
# statistical data visualization; https://seaborn.pydata.org/

# Load the dataset
file_path = 'fer2013.csv'  # relative or absolute path to the dataset
data = pd.read_csv(file_path, on_bad_lines='skip') # skip bad lines if any

data['Usage'] = data['Usage'].astype(str).str.strip() # remove the leading/trailing spaces from the 'Usage' column

# 
train_data = data[data['Usage'] == 'Training'] # backup the original data
public_test_data = data[data['Usage'] == 'PublicTest']  # had problems with the 'PublicTest' data
test_data = data[data['Usage'] == 'PrivateTest'] # had problems with the 'PrivateTest' data

print("Training data sample:", train_data.head()) # make sure the data is loaded correctly
print("Public test data sample:", public_test_data.head()) # ...
print("Private test data sample:", test_data.head()) # ...

def prepare_data(df, expected_pixels=2304):  # 48x48 images have 2304 pixels
    pixel_arrays = [] # store the pixel arrays
    emotions = [] # store the emotions
    for _, row in df.iterrows(): # iterate over the rows of the dataframe
        pixel_list = np.fromstring(row['pixels'], dtype=int, sep=' ')
        if len(pixel_list) == expected_pixels:
            pixel_arrays.append(pixel_list)
            emotions.append(row['emotion'])
    X = np.array(pixel_arrays)
    y = np.array(emotions)
    return X, y

print("Unique values in 'Usage' column:", data['Usage'].unique()) # check the unique values in the 'Usage' column

train_data = data[data['Usage'].str.strip().eq('Training')]
test_data = data[data['Usage'].str.strip().eq('PrivateTest')]

X_train, y_train = prepare_data(train_data)
X_test, y_test = prepare_data(test_data)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

svm_model = SVC(kernel='rbf')
svm_model.fit(X_train, y_train)

y_pred = svm_model.predict(X_test)

print("Accuracy:", svm_model.score(X_test, y_test))
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Training data sample:    emotion                                             pixels     Usage
0        0  70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...  Training
1        0  151 150 147 155 148 133 111 140 170 174 182 15...  Training
2        2  231 212 156 164 174 138 161 173 182 200 106 38...  Training
3        4  24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...  Training
4        6  4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...  Training
Public test data sample:        emotion                                             pixels       Usage
28709        0  254 254 254 254 254 249 255 160 2 58 53 70 77 ...  PublicTest
28710        1  156 184 198 202 204 207 210 212 213 214 215 21...  PublicTest
28711        4  69 118 61 60 96 121 103 87 103 88 70 90 115 12...  PublicTest
28712        6  205 203 236 157 83 158 120 116 94 86 155 180 2...  PublicTest
28713        3  87 79 74 66 74 96 77 80 80 84 83 89 102 91 84 ...  PublicTest
Private test data sample:        emotion             

2. Train a support vector machine using the Action Units of labeled samples from phoebe_AU.csv. Use 5-fold cross-validation on this training set to report the performance. Report your perceived qualitative performance on the unknown labels (e.g. How many appear correct? Provide your own labels as unknown groundtruth to help quantify your results.)

In [26]:
from sklearn.model_selection import cross_val_score, KFold

data = pd.read_csv('phoebe_AU.csv')

X = data.iloc[:, 1:-1]  # exclude the first and last columns
y = data.iloc[:, -1]   # we expect the last column to be the target

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

svm_model = SVC(kernel='rbf') # default kernel is 'rbf' and is faster than 'linear' kernel

kf = KFold(n_splits=5, shuffle=True)

cv_scores = cross_val_score(svm_model, X_scaled, y, cv=kf, scoring='accuracy')
print(f"Average 5-Fold CV Score: {cv_scores.mean()}")

svm_model.fit(X_scaled, y)
predicted_labels = svm_model.predict(X_scaled) # compare the predicted labels to the actual labels

for actual, predicted in zip(y, predicted_labels):
    print(f"Actual: {actual}, Predicted: {predicted}")

print(classification_report(y, predicted_labels))
print(confusion_matrix(y, predicted_labels))

Average 5-Fold CV Score: 0.3968421052631579
Actual: unknown, Predicted: surprise
Actual: angry, Predicted: sad
Actual: surprise, Predicted: surprise
Actual: happy, Predicted: happy
Actual: unknown, Predicted: unknown
Actual: unknown, Predicted: unknown
Actual: angry, Predicted: surprise
Actual: surprise, Predicted: surprise
Actual: happy, Predicted: happy
Actual: happy, Predicted: happy
Actual: happy, Predicted: happy
Actual: unknown, Predicted: happy
Actual: happy, Predicted: happy
Actual: unknown, Predicted: unknown
Actual: happy, Predicted: happy
Actual: happy, Predicted: happy
Actual: sad, Predicted: sad
Actual: happy, Predicted: happy
Actual: happy, Predicted: happy
Actual: happy, Predicted: happy
Actual: happy, Predicted: happy
Actual: sad, Predicted: sad
Actual: sad, Predicted: sad
Actual: sad, Predicted: sad
Actual: sad, Predicted: sad
Actual: angry, Predicted: angry
Actual: angry, Predicted: angry
Actual: happy, Predicted: happy
Actual: happy, Predicted: happy
Actual: happy, P

After analyzing the model, it seems that it was quite accurate in determining the emotion conveyed for clearly labeled images, while it was still quite uncertain for images labeled as 'unknown'. Based on my own labels as unknown groundtruths, most of the expressions are mixes of angry or sad facial features, but this is not matched with the model. 

Part B - Neural Network

1. Neural Network. Train a neural network using the images from fer2013.csv using Keras. Your first layer should be a Conv2D layer, and the last layers should be a Dense layer followed by a Softmax. Use the Training set for training, PublicTest validation set to avoid overfitting, and the PrivateTest test set for testing. Aim for a minimum validation accuracy of 40% on the Fer2013 validation set. To enhance your model's performance, experiment with various batch sizes and epochs. Incorporate dropout and normalization techniques to further mitigate overfitting and improve generalization. Report precision, recall, accuracy, F1 score, and create a confusion matrix on the test set.

In [12]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

file_path = 'fer2013.csv'
data = pd.read_csv(file_path)

def prepare_data(df):
    X = np.array([np.fromstring(pixels, dtype=float, sep=' ').reshape(48, 48, 1) for pixels in df['pixels']]) # 48x48 images with 1 channel (required by the Conv2D layer)
    X = X / 255.0  # normalize the pixel values
    y = to_categorical(df['emotion'].values)
    return X, y

train_data = data[data['Usage'] == 'Training']
val_data = data[data['Usage'] == 'PublicTest']
test_data = data[data['Usage'] == 'PrivateTest']
X_train, y_train = prepare_data(train_data)
X_val, y_val = prepare_data(val_data)
X_test, y_test = prepare_data(test_data)

model = Sequential([
    Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Flatten(),
    Dense(1024, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(7, activation='softmax')  # 7 is the number of emotion labels
])

# boilerplate model code

model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

datagen = ImageDataGenerator(
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True)

history = model.fit(datagen.flow(X_train, y_train, batch_size=64),
                    epochs=5,
                    verbose=1,
                    validation_data=(X_val, y_val))

test_loss, test_acc = model.evaluate(X_test, to_categorical(test_data['emotion'].values))
print(f'Test accuracy: {test_acc * 100:.2f}%')

y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(to_categorical(test_data['emotion'].values), axis=1)

print(classification_report(y_true, y_pred_classes))

print(confusion_matrix(y_true, y_pred_classes))

c:\Users\Ritchie\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/5


c:\Users\Ritchie\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


449/449 ━━━━━━━━━━━━━━━━━━━━ 141s 310ms/step - accuracy: 0.2351 - loss: 2.5274 - val_accuracy: 0.2839 - val_loss: 1.8768
Epoch 2/5
449/449 ━━━━━━━━━━━━━━━━━━━━ 130s 290ms/step - accuracy: 0.3512 - loss: 1.7546 - val_accuracy: 0.2973 - val_loss: 1.9670
Epoch 3/5
449/449 ━━━━━━━━━━━━━━━━━━━━ 131s 291ms/step - accuracy: 0.3816 - loss: 1.6064 - val_accuracy: 0.3335 - val_loss: 1.9096
Epoch 4/5
449/449 ━━━━━━━━━━━━━━━━━━━━ 129s 286ms/step - accuracy: 0.4063 - loss: 1.5435 - val_accuracy: 0.4202 - val_loss: 1.6558
Epoch 5/5
449/449 ━━━━━━━━━━━━━━━━━━━━ 127s 282ms/step - accuracy: 0.4384 - loss: 1.4820 - val_accuracy: 0.4386 - val_loss: 1.5215
113/113 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - accuracy: 0.4213 - loss: 1.5542
Test accuracy: 41.99%
113/113 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step
              precision    recall  f1-score   support

           0       0.36      0.25      0.30       491
           1       0.00      0.00      0.00        55
           2       0.31      0.17      0.22       52

c:\Users\Ritchie\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Ritchie\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Ritchie\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

Test accuracy: 41.99%

              precision    recall  f1-score   support

           0       0.36      0.25      0.30       491
           1       0.00      0.00      0.00        55
           2       0.31      0.17      0.22       528
           3       0.44      0.91      0.60       879
           4       0.30      0.37      0.33       594
           5       0.75      0.43      0.55       416
           6       0.57      0.14      0.23       626

    accuracy                           0.42      3589
   macro avg       0.39      0.33      0.32      3589
weighted avg       0.44      0.42      0.38      3589
...
confusion matrix
 [[ 19   0  18 802  33   2   5]
 [ 55   0  42 255 222   5  15]
 [ 28   0  67 108  25 178  10]
 [ 50   0  29 252 198   7  90]]


2. Test. Use your trained neural network from Part B.1 and classify the Phoebe unknown image data. Report your perceived performance on the unknown labels, comparing it to the SVM in Part A.2.

In [15]:
from PIL import Image
from tensorflow.keras.models import load_model

# load the dataset to get the unknown images
data = pd.read_csv('phoebe_AU.csv')
unknown_data = data[data['label'] == 'unknown']

model.save('cnn_b1.h5')

model = load_model('cnn_b1.h5')  # load the model file

def preprocess_image(image_path):
    img = Image.open(image_path).convert('L')  # convert to grayscale
    img = img.resize((48, 48))  # resize to match the input shape expected by the model
    img_array = np.array(img)
    img_array = img_array / 255.0
    img_array = img_array.reshape(1, 48, 48, 1)
    return img_array

results = []
for filename in unknown_data['file_name']:
    image_path = f'images/unknown/{filename}' # assuming the images are in a folder called 'images'
    img_array = preprocess_image(image_path)
    prediction = model.predict(img_array)
    predicted_label = np.argmax(prediction)  # assuming the labels are encoded as integers
    results.append((filename, predicted_label))

# Display the results
for filename, label in results:
    print(f'Image: {filename}, Predicted label: {label}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Image: 1_01.jpg, Predicted label: 4
Image: 4_01.jpg, Predicted label: 4
Image: 4_20.jpg, Predicted label: 4
Image: 8_01.jpg, Predicted label: 3
Image: 9_41.jpg, Predicted label: 3
Image: 26_123.jpg, Predicted label: 3
Image: 35_42.jpg, Predicted label: 4
Image: 41_06.jpg, Predicted label: 3
Image: 44_01.jpg, Predicted label: 4
Image: 46_03.jpg, Predicted label: 3
Image: 48_01.jpg, Predicted label: 4
Image: 52_31.jpg, Predicted label: 3


Using the legend from the dataset: (0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral), and comparing with the images given, the model is more accurate and decisive with an emotion when compared to the SVM in Part A.2. It is so because it is giving definite and not unknown classifications for each and every image, and is mostly accurate based on my perceived performance. It seems to have misaligned with 52_31.jpg, where Phoebe looks more on the sadder side, but was labeled as happy.

3. Fine-tune the Neural Network, and re-classify. Fine-tune your neural network on the Phoebe-face image dataset provided (Hints: use imread() in grayscale to read the images, and freeze early layer weights during fine-tuning). Then, reclassify the images in unknown. Do you think the results improved compared to Part B.2?

In [25]:
import os
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from PIL import Image


def rgb2gray(rgb):
    return np.dot(rgb[..., :3], [0.2989, 0.5870, 0.1140]) # convert to grayscale using the formula

emotions = ['angry', 'disgust', 'surprise', 'happy', 'sad']  # emotions in the dataset
image_data = []
labels = []

for i, emotion in enumerate(emotions):
    emotion_dir = f'images/{emotion}/'
    for file in os.listdir(emotion_dir):
        # some files may not be images
        if not file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
            continue

        img_path = os.path.join(emotion_dir, file)
        img = Image.open(img_path).convert('L')  # convert to grayscale
        img = img.resize((48, 48))  # resize to 48x48 to match the input shape expected by the model
        img_array = np.array(img) / 255.0
        image_data.append(img_array.reshape(48, 48, 1))
        labels.append(i)

X_phoebe = np.array(image_data)
y_phoebe = to_categorical(np.array(labels), num_classes=len(emotions))

X_phoebe_train, X_phoebe_val, y_phoebe_train, y_phoebe_val = train_test_split(X_phoebe, y_phoebe, test_size=0.2)

model = load_model('cnn_b1.h5') # load the model file

# fine-tune the model by removing the last layer and adding a new output layer. this is needed because the number of classes is different
base_model = model  # store the original model
base_model.layers.pop()  # remove the last laye
new_output = Dense(len(emotions), activation='softmax')(base_model.layers[-1].output) # add a new output layer
new_model = Model(inputs=base_model.inputs, outputs=new_output) # create a new model with the modified output

# freeze the first 5 layers
for layer in new_model.layers[:5]:
    layer.trainable = False

# try a smaller learning rate to avoid overfitting
new_model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])

# fine-tune the model by training on the new dataset
history = new_model.fit(X_phoebe_train, y_phoebe_train, epochs=5, validation_data=(X_phoebe_val, y_phoebe_val))

# Save the fine-tuned model
new_model.save('cnn_b3.h5')

# load the dataset to get the unknown images
data = pd.read_csv('phoebe_AU.csv')
unknown_data = data[data['label'] == 'unknown']

def preprocess_image(image_path):
    img = Image.open(image_path).convert('L')
    img = img.resize((48, 48))
    img_array = np.array(img) / 255.0
    return img_array.reshape(1, 48, 48, 1)

# perform the classification
results = []
for filename in unknown_data['file_name']:
    image_path = f'images/unknown/{filename}'
    img_array = preprocess_image(image_path)
    prediction = new_model.predict(img_array)
    predicted_label = np.argmax(prediction)
    results.append((filename, emotions[predicted_label]))  # convert the label back to the original emotion

# show the results
for filename, label in results:
    print(f'Image: {filename}, Predicted label: {label}')


Epoch 1/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 221ms/step - accuracy: 0.1054 - loss: 1.6157 - val_accuracy: 0.1111 - val_loss: 1.6463
Epoch 2/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step - accuracy: 0.1088 - loss: 1.6148 - val_accuracy: 0.1111 - val_loss: 1.6414
Epoch 3/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step - accuracy: 0.1048 - loss: 1.6330 - val_accuracy: 0.1111 - val_loss: 1.6372
Epoch 4/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step - accuracy: 0.0898 - loss: 1.6022 - val_accuracy: 0.0556 - val_loss: 1.6334
Epoch 5/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step - accuracy: 0.1472 - loss: 1.6103 - val_accuracy: 0.0556 - val_loss: 1.6305


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Image: 1_01.jpg, Predicted label: angry
Image: 4_01.jpg, Predicted label: angry
Image: 4_20.jpg, Predicted label: angry
Image: 8_01.jpg, Predicted label: disgust
Image: 9_41.jpg, Predicted label: disgust
Image: 26_123.jpg, Predicted label: disgust
Image: 35_42.jpg, Predicted label: angry
Image: 41_06.jpg, Predicted label: disgust
Image: 44_01.jpg, Predicted label: disgust
Image: 46_03.jpg, Predicted label: disgust
Image: 48_01.jpg, Predicted label: disgust
Image: 52_31.jpg, Predicted label: disgust


The model seems to have almost the same performance when compared to Part B.2 as there are only 2 main emotions presented in the model predictions; angry and disgust. This may be part in due to the fact that the original neural network had 7 dense layers (due to the varied different emotions from the fer2013 dataset), but this newly trained neural network has been condensed down to 5 dense layers to account for the phoebe and Action Units dataset, which may be outputting a more generalized performance.

Part C - Comparison between Methods

1. Compare. Compare the results from the 4 models (SVM-Fer2013, SVM-OpenFace, NN-Fer2013, NN-FineTuned) on the Phoebe unknown dataset. Specifically compare the approach with hand-crafted features (SVM-OpenFace) versus neural network extracted features (NN-FineTuned). Choose the one that you think worked best with this dataset. Justify your answer based on the results from Part A and Part B and discuss limitations.

In Part C, it becomes evident that the neural network finetuned model outperforms on the Phoebe dataset due to the variation in dense output layers of the network itself. The SVM opened-faced model falls short in capturing the nuanced emotional expressions compared to the neutral network because of the dense and KFold parameters.